In [1]:
import pandas as pd
import numpy as np
import glob
import os

First, we are going to create 1 metadata file, with all light curves. I have 3 metada files that are relevant:

1.- bts_sn_host_meta.csv: contains metadata of all sn light curves found in BTS that have host z information.

2.- tns_search_meta.csv: contains metadata of all lightcurves found in the TNS. Most objects are duplicated, as they exist in BTS too. However, the metadata available is more extensive, so it makes sense to crossmatch.

3.- yse_not_in_bts_or_tns.csv: contains lightcurve metadata found in the YSE dataset, that is not found in the previous 2 files.

The first step is to merge file 1 and file 2, getting rid of duplicates

In [19]:
bts_meta = pd.read_csv('../data/real/bts_sn_host_meta.csv')
print(bts_meta.shape)
bts_meta.head()

(5159, 17)


,ZTFID,IAUID,RA,Dec,peakt,peakfilt,peakmag,peakabs,duration,rise,fade,type,redshift,hostabs,hostcol,b,A_V
0,ZTF17aabtvsy,SN2022yei,10:35:32.09,+37:38:59.0,1870.99,r,18.0303,-19.41,>34.229,>6.01,28.219,SN Ia,0.06922,-22.70,1.06,59.641962,0.053
1,ZTF17aadlxmv,SN2020adv,08:29:47.59,+33:54:22.8,879.69,g,17.9475,-19.34,25.146,10.951,14.195,SN Ia,0.06200,-22.16,0.91,34.174702,0.106
2,ZTF18aaaibml,SN2020buc,10:08:31.27,+09:14:23.0,909.80,r,18.2606,-17.70,69.73,27.302,42.428,SN II,0.03480,-19.67,0.38,47.637802,0.092
3,ZTF18aaaonon,SN2022jjs,10:19:05.51,+14:24:16.6,1703.77,g,18.5663,-19.08,>18.631,3.691,>14.94,SN Ia,0.07141,-21.67,1.10,52.363911,0.150
4,ZTF18aaaqexr,SN2020chl,11:43:06.37,+15:34:01.7,891.92,g,18.4475,-19.10,17.228,5.46,11.768,SN Ia,0.07020,-22.72,0.97,70.370720,0.101


In [20]:
tns_meta = pd.read_csv('../data/real/tns_search_meta.csv')
print(tns_meta.shape)
tns_meta.head()

(6335, 26)


,ID,Name,RA,DEC,Obj. Type,Redshift,Host Name,Host Redshift,Reporting Group/s,Discovery Data Source/s,...,Public,End Prop. Period,Discovery Mag/Flux,Discovery Filter,Discovery Date (UT),Sender,Remarks,Discovery Bibcode,Classification Bibcodes,Ext. catalog/s
0,24959,SN 2018mg,10:56:53.580,+79:16:42.50,SN Ia,0.110,NaN,NaN,ZTF,ZTF,...,1,2018-02-07,20.1,g-ZTF,2018-01-11 10:52:26.000,ZTF_Bot1,NaN,2018TNSTR.172....1L,2018TNSCR.175....1F,NaN
1,24958,SN 2018mf,10:22:09.778,+10:26:24.09,SN Ia,0.107,NaN,NaN,"ZTF, Pan-STARRS","ZTF, Pan-STARRS",...,1,NaN,19.6,g-ZTF,2018-01-14 10:29:02.000,ZTF_Bot1,NaN,2018TNSTR.172....1L,2018TNSCR.175....1F,NaN
2,25295,SN 2018yt,08:59:35.590,+45:36:41.90,SN II,0.052,NaN,NaN,ZTF,ZTF,...,1,2018-04-24,18.9,r-ZTF,2018-02-09 06:00:00.000,ZTF_Bot1,NaN,2018TNSTR.544....1T,2018TNSCR.548....1T,NaN
3,26057,SN 2018bbk,07:52:26.950,+16:29:38.50,SN II,0.038,NaN,NaN,ZTF,ZTF,...,1,NaN,18.7,r-ZTF,2018-03-05 03:36:00.000,ZTF_Bot1,NaN,2018TNSTR.543....1F,2018TNSCR.549....1G,NaN
4,25459,SN 2018aex,11:00:45.380,+22:17:14.96,SN II,0.023,NaN,0.0229,"ZTF, GaiaAlerts, ATLAS","ZTF, GaiaAlerts, ATLAS",...,1,NaN,18.1,r-ZTF,2018-03-05 06:07:12.000,ZTF_Bot1,NaN,2018TNSTR.545....1L,2018TNSCR.547....1N,NaN


In [21]:
print(bts_meta.keys())
print(tns_meta.keys())

Index(['ZTFID', 'IAUID', 'RA', 'Dec', 'peakt', 'peakfilt', 'peakmag',
       'peakabs', 'duration', 'rise', 'fade', 'type', 'redshift', 'hostabs',
       'hostcol', 'b', 'A_V'],
      dtype='object')
Index(['ID', 'Name', 'RA', 'DEC', 'Obj. Type', 'Redshift', 'Host Name',
       'Host Redshift', 'Reporting Group/s', 'Discovery Data Source/s',
       'Classifying Group/s', 'Associated Group/s', 'Disc. Internal Name',
       'Disc. Instrument/s', 'Class. Instrument/s', 'TNS AT', 'Public',
       'End Prop. Period', 'Discovery Mag/Flux', 'Discovery Filter',
       'Discovery Date (UT)', 'Sender', 'Remarks', 'Discovery Bibcode',
       'Classification Bibcodes', 'Ext. catalog/s'],
      dtype='object')


Clearly some keys are repeated. We'll rename the keys so the ones duplicated get dropped later on merge

In [22]:
rename_dict={'Name':'IAUID',
 'DEC':'Dec',
 'Obj. Type':'type',
 'Redshift':'redshift',
 'Host Name':'hostname',
 'Host Redshift':'hostredshift',
 'Reporting Group/s':'reporting_groups',
 'Discovery Data Source/s':'data_sources',
 'Classifying Group/s':'classifying_groups',
 'Associated Group/s':'associated_groups',
 'Disc. Internal Name':'ZTFID',
 }

In [23]:
tns_meta.rename(columns=rename_dict,inplace=True)

In [25]:
tns_meta.keys()

Index(['ID', 'IAUID', 'RA', 'Dec', 'type', 'redshift', 'hostname',
       'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID',
       'Disc. Instrument/s', 'Class. Instrument/s', 'TNS AT', 'Public',
       'End Prop. Period', 'Discovery Mag/Flux', 'Discovery Filter',
       'Discovery Date (UT)', 'Sender', 'Remarks', 'Discovery Bibcode',
       'Classification Bibcodes', 'Ext. catalog/s'],
      dtype='object')

In [26]:
relevant_columns = ['IAUID','RA','Dec','type','redshift','hostname','hostredshift','reporting_groups','data_sources','classifying_groups','associated_groups','ZTFID']
tns_meta=tns_meta[relevant_columns]
tns_meta.keys()

Index(['IAUID', 'RA', 'Dec', 'type', 'redshift', 'hostname', 'hostredshift',
       'reporting_groups', 'data_sources', 'classifying_groups',
       'associated_groups', 'ZTFID'],
      dtype='object')

now we merge both data frames

In [27]:
bts_tns_merge_meta = pd.merge(tns_meta,bts_meta,how='outer',on='ZTFID')

In [28]:
bts_tns_merge_meta.shape

(7690, 28)

In [29]:
bts_tns_merge_meta.keys()

Index(['IAUID_x', 'RA_x', 'Dec_x', 'type_x', 'redshift_x', 'hostname',
       'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'IAUID_y', 'RA_y',
       'Dec_y', 'peakt', 'peakfilt', 'peakmag', 'peakabs', 'duration', 'rise',
       'fade', 'type_y', 'redshift_y', 'hostabs', 'hostcol', 'b', 'A_V'],
      dtype='object')

In [16]:
bts_tns_merge_meta.head()

,IAUID_x,Dec_x,type_x,redshift_x,hostname,hostredshift,reporting_groups,data_sources,classifying_groups,associated_groups,...,peakabs,duration,rise,fade,type_y,redshift_y,hostabs,hostcol,b,A_V
0,SN 2018mg,+79:16:42.50,SN Ia,0.110,NaN,NaN,ZTF,ZTF,ZTF,ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SN 2018mf,+10:26:24.09,SN Ia,0.107,NaN,NaN,"ZTF, Pan-STARRS","ZTF, Pan-STARRS",ZTF,Pan-STARRS; ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SN 2018yt,+45:36:41.90,SN II,0.052,NaN,NaN,ZTF,ZTF,ZTF,ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SN 2018bbk,+16:29:38.50,SN II,0.038,NaN,NaN,ZTF,ZTF,ZTF,ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SN 2018aex,+22:17:14.96,SN II,0.023,NaN,0.0229,"ZTF, GaiaAlerts, ATLAS","ZTF, GaiaAlerts, ATLAS",ZTF,ATLAS; GaiaAlerts; ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#let's keep only one Dec column
bts_tns_merge_meta['Dec'] = bts_tns_merge_meta['Dec_x'].combine_first(bts_tns_merge_meta['Dec_y'])
bts_tns_merge_meta = bts_tns_merge_meta.drop(columns=['Dec_x', 'Dec_y'])
print(bts_tns_merge_meta.keys())
bts_tns_merge_meta.head()

Index(['IAUID_x', 'RA_x', 'type_x', 'redshift_x', 'hostname', 'hostredshift',
       'reporting_groups', 'data_sources', 'classifying_groups',
       'associated_groups', 'ZTFID', 'IAUID_y', 'RA_y', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'type_y',
       'redshift_y', 'hostabs', 'hostcol', 'b', 'A_V', 'Dec'],
      dtype='object')


,IAUID_x,RA_x,type_x,redshift_x,hostname,hostredshift,reporting_groups,data_sources,classifying_groups,associated_groups,...,duration,rise,fade,type_y,redshift_y,hostabs,hostcol,b,A_V,Dec
0,SN 2018mg,10:56:53.580,SN Ia,0.110,NaN,NaN,ZTF,ZTF,ZTF,ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+79:16:42.50
1,SN 2018mf,10:22:09.778,SN Ia,0.107,NaN,NaN,"ZTF, Pan-STARRS","ZTF, Pan-STARRS",ZTF,Pan-STARRS; ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+10:26:24.09
2,SN 2018yt,08:59:35.590,SN II,0.052,NaN,NaN,ZTF,ZTF,ZTF,ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+45:36:41.90
3,SN 2018bbk,07:52:26.950,SN II,0.038,NaN,NaN,ZTF,ZTF,ZTF,ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+16:29:38.50
4,SN 2018aex,11:00:45.380,SN II,0.023,NaN,0.0229,"ZTF, GaiaAlerts, ATLAS","ZTF, GaiaAlerts, ATLAS",ZTF,ATLAS; GaiaAlerts; ZTF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+22:17:14.96


In [33]:
#let's repeat that for all repeated columns
#RA
# bts_tns_merge_meta['RA'] = bts_tns_merge_meta['RA_x'].combine_first(bts_tns_merge_meta['RA_y'])
# bts_tns_merge_meta = bts_tns_merge_meta.drop(columns=['RA_x', 'RA_y'])
# print(bts_tns_merge_meta.keys())
# bts_tns_merge_meta.head()

#IAUID
# bts_tns_merge_meta['IAUID'] = bts_tns_merge_meta['IAUID_x'].combine_first(bts_tns_merge_meta['IAUID_y'])
# bts_tns_merge_meta = bts_tns_merge_meta.drop(columns=['IAUID_x', 'IAUID_y'])
# print(bts_tns_merge_meta.keys())
# bts_tns_merge_meta.head()

#type
# bts_tns_merge_meta['type'] = bts_tns_merge_meta['type_x'].combine_first(bts_tns_merge_meta['type_y'])
# bts_tns_merge_meta = bts_tns_merge_meta.drop(columns=['type_x', 'type_y'])
# print(bts_tns_merge_meta.keys())
# bts_tns_merge_meta.head()

#redshift
bts_tns_merge_meta['redshift'] = bts_tns_merge_meta['redshift_x'].combine_first(bts_tns_merge_meta['redshift_y'])
bts_tns_merge_meta = bts_tns_merge_meta.drop(columns=['redshift_x', 'redshift_y'])
print(bts_tns_merge_meta.keys())
bts_tns_merge_meta.head()

Index(['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'Dec', 'RA', 'IAUID', 'type', 'redshift'],
      dtype='object')


,hostname,hostredshift,reporting_groups,data_sources,classifying_groups,associated_groups,ZTFID,peakt,peakfilt,peakmag,...,fade,hostabs,hostcol,b,A_V,Dec,RA,IAUID,type,redshift
0,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaaoktp,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,+79:16:42.50,10:56:53.580,SN 2018mg,SN Ia,0.110
1,NaN,NaN,"ZTF, Pan-STARRS","ZTF, Pan-STARRS",ZTF,Pan-STARRS; ZTF,ZTF18aaaoaeq,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,+10:26:24.09,10:22:09.778,SN 2018mf,SN Ia,0.107
2,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaayemw,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,+45:36:41.90,08:59:35.590,SN 2018yt,SN II,0.052
3,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaewugy,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,+16:29:38.50,07:52:26.950,SN 2018bbk,SN II,0.038
4,NaN,0.0229,"ZTF, GaiaAlerts, ATLAS","ZTF, GaiaAlerts, ATLAS",ZTF,ATLAS; GaiaAlerts; ZTF,ZTF18aabssth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,+22:17:14.96,11:00:45.380,SN 2018aex,SN II,0.023


In [34]:
#let's get rid of spaces in some IAUID
bts_tns_merge_meta['IAUID'] = bts_tns_merge_meta['IAUID'].str.replace(' ', '', regex=False)

In [35]:
bts_tns_merge_meta.shape

(7690, 23)

In [36]:
bts_tns_merge_meta.ZTFID.unique().shape

(7690,)

Good, there doesn't seem to be any repeated objects. Now, let's load the YSE metadata file and merge it with the df we just created.
There will be some inconsistencies that we'll need to deal with:

1.-fix column names

2.-RA/DEC are in different units

3.-types have different nomenclatures and there are some sub-types

In [37]:
yse_meta = pd.read_csv('../data/real/yse_not_in_bts_or_tns.csv')
print(yse_meta.keys())
yse_meta.shape

Index(['Object', 'RA', 'Dec', 'Spec. Class', 'Redshift, $z$', 'Prediction',
       'Confidence', 'p_SNII', 'p_SNIa', 'p_SNIbc', 'IAUID'],
      dtype='object')


(1808, 11)

In [38]:
yse_rename_dict={'Spec. Class':'type_x','Redshift, $z$':'redshift','Prediction':'type_y'}
yse_meta.rename(columns=yse_rename_dict,inplace=True)
yse_meta.keys()

Index(['Object', 'RA', 'Dec', 'type_x', 'redshift', 'type_y', 'Confidence',
       'p_SNII', 'p_SNIa', 'p_SNIbc', 'IAUID'],
      dtype='object')

In [47]:
#let's keep only the objects that have a prediction of at least 50%confidence
yse_meta=yse_meta[yse_meta['Confidence']>=0.5]
yse_meta.shape

(1701, 11)

In [50]:
#now, let's keep the spec type if it exists, if not, keep the prediction
#yse_meta['type'] = yse_meta['type_x'].combine_first(yse_meta['type_y'])
#yse_meta = yse_meta.drop(columns=['type_x', 'type_y'])
print(yse_meta.keys())
yse_meta.shape

Index(['Object', 'RA', 'Dec', 'redshift', 'Confidence', 'p_SNII', 'p_SNIa',
       'p_SNIbc', 'IAUID', 'type'],
      dtype='object')


(1701, 10)

In [54]:
bts_tns_merge_meta.shape

(7690, 23)

In [52]:
#now let's merge the dfs:
merge_meta = pd.merge(bts_tns_merge_meta,yse_meta,how='outer',on='IAUID')

In [53]:
merge_meta.shape

(9300, 32)

So... even if ZTFIDs are unique, some IAUIDs are repated. but if I then check those, that also gives me some repeated objects?

In [60]:
y=yse_meta.IAUID.unique()
b=bts_tns_merge_meta.IAUID.unique()
print(y.shape)
print(b.shape)

(1701,)
(7673,)


In [63]:
len(set(np.concatenate((y,b))))

9283

In [64]:
merge_meta.IAUID.unique().shape

(9283,)

let's see which ones are repeated

In [69]:
value_counts = merge_meta['IAUID'].value_counts()
repeated = value_counts[value_counts>1]

In [77]:
rep=list(repeated.index)

In [78]:
rep

['SN2020byb',
 'SN2019uzd',
 'SN2020lsj',
 'SN2023uqw',
 'SN2020beu',
 'SN2019pkg',
 'SN2022zkb',
 'SN2019zrs',
 'SN2021pkq',
 'SN2019cww',
 'SN2020yvs',
 'SN2020pnn',
 'SN2021oud',
 'SN2023jlx',
 'SN2019bue',
 'SN2020ywo',
 'SN2018ert']

In [83]:
merge_meta.keys()

Index(['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'Dec_x', 'RA_x', 'IAUID', 'type_x', 'redshift_x', 'Object',
       'RA_y', 'Dec_y', 'redshift_y', 'Confidence', 'p_SNII', 'p_SNIa',
       'p_SNIbc', 'type_y'],
      dtype='object')

In [84]:
merge_meta[merge_meta.IAUID.isin(rep)][['ZTFID','IAUID']]

,ZTFID,IAUID
145,ZTF18aaoxryq,SN2018ert
146,ZTF18ablrljh,SN2018ert
665,ZTF19aammlzn,SN2019bue
666,ZTF19aamgttu,SN2019bue
760,ZTF18aathacy,SN2019cww
761,ZTF19aaqtyku,SN2019cww
1276,ZTF18abqlykl,SN2019pkg
1277,ZTF19abwswzb,SN2019pkg
1589,ZTF19aaetsvm,SN2019uzd
1590,ZTF22abaucrh,SN2019uzd


for whatever reason, some objst have different ztf_id but share iau_id. if I search for them on lasair, it shows the same images and metadata, but different light curves. I'll merge them. 

In [91]:
repeated_iauid=merge_meta[merge_meta.IAUID.isin(rep)][['IAUID']].IAUID.unique()

In [92]:
repeated_iauid

array(['SN2018ert', 'SN2019bue', 'SN2019cww', 'SN2019pkg', 'SN2019uzd',
       'SN2019zrs', 'SN2020beu', 'SN2020byb', 'SN2020lsj', 'SN2020pnn',
       'SN2020yvs', 'SN2020ywo', 'SN2021oud', 'SN2021pkq', 'SN2022zkb',
       'SN2023jlx', 'SN2023uqw'], dtype=object)

In [93]:
rep_merge=pd.DataFrame(columns=merge_meta.columns)
for id_value in repeated_iauid:
    subset=merge_meta[merge_meta.IAUID==id_value]
    combine_row=subset.iloc[0].combine_first(subset.iloc[1])
    rep_merge=rep_merge.append(combine_row,ignore_index=True)

In [99]:
rep_merge

,hostname,hostredshift,reporting_groups,data_sources,classifying_groups,associated_groups,ZTFID,peakt,peakfilt,peakmag,...,redshift_x,Object,RA_y,Dec_y,redshift_y,Confidence,p_SNII,p_SNIa,p_SNIbc,type_y
0,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaoxryq,346.68,r,18.8561,...,0.094000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF19aammlzn,567.69,g,16.8443,...,0.030800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"ZTF, GaiaAlerts, Pan-STARRS, ATLAS","ZTF, GaiaAlerts, Pan-STARRS, ATLAS",ZTF,ATLAS; GaiaAlerts; Pan-STARRS; ZTF,ZTF18aathacy,592.76,r,16.8281,...,0.023400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,"ZTF, ATLAS, Pan-STARRS","ZTF, ATLAS, Pan-STARRS",ZTF,ATLAS; Pan-STARRS; ZTF,ZTF18abqlykl,748.68,g,17.0887,...,0.041000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NGC 7427,0.032370,ALeRCE,ZTF,ePESSTO+,ALeRCE,ZTF19aaetsvm,1810.80,r,17.1518,...,0.032000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF20aaawrja,977.73,r,18.4383,...,0.065200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF19aawmcjt,1000.83,r,17.6796,...,0.041000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,"ZTF, ATLAS","ZTF, ATLAS",ZTF,ATLAS; ZTF,ZTF18aarvjhs,895.03,r,18.1965,...,0.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,"SGLF, ATLAS, ZTF, Pan-STARRS","ZTF, ATLAS, Pan-STARRS",ZTF,ATLAS; Pan-STARRS; ZTF,ZTF20abdxuew,1014.77,r,18.6701,...,0.070000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,ZTF,ZTF,NaN,ZTF,ZTF18aavwurv,1053.80,r,17.5688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
merge_meta_0=merge_meta[~merge_meta['IAUID'].isin(repeated_iauid)]
merge_meta_0.shape

(9266, 32)

I got rid of the repeated objects in the big df and now I'll add the combined repeated objects that I just created

In [102]:
merge_meta_1 = pd.concat([merge_meta_0,rep_merge],ignore_index=True)

In [103]:
merge_meta_1.shape

(9283, 32)

now I need to clean up the columns

In [104]:
merge_meta_1.keys()

Index(['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'Dec_x', 'RA_x', 'IAUID', 'type_x', 'redshift_x', 'Object',
       'RA_y', 'Dec_y', 'redshift_y', 'Confidence', 'p_SNII', 'p_SNIa',
       'p_SNIbc', 'type_y'],
      dtype='object')

In [105]:
#Dec, 
merge_meta_1['Dec'] = merge_meta_1['Dec_x'].combine_first(merge_meta_1['Dec_y'])
merge_meta_1 = merge_meta_1.drop(columns=['Dec_x', 'Dec_y'])
print(merge_meta_1.keys())


Index(['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'RA_x', 'IAUID', 'type_x', 'redshift_x', 'Object', 'RA_y',
       'redshift_y', 'Confidence', 'p_SNII', 'p_SNIa', 'p_SNIbc', 'type_y',
       'Dec'],
      dtype='object')


In [108]:
#RA
merge_meta_1['RA'] = merge_meta_1['RA_x'].combine_first(merge_meta_1['RA_y'])
merge_meta_1 = merge_meta_1.drop(columns=['RA_x', 'RA_y'])
print(merge_meta_1.keys())

Index(['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'IAUID', 'type_x', 'redshift_x', 'Object', 'redshift_y',
       'Confidence', 'p_SNII', 'p_SNIa', 'p_SNIbc', 'type_y', 'Dec', 'RA'],
      dtype='object')


In [109]:
#redshift
merge_meta_1['redshift'] = merge_meta_1['redshift_x'].combine_first(merge_meta_1['redshift_y'])
merge_meta_1 = merge_meta_1.drop(columns=['redshift_x', 'redshift_y'])
print(merge_meta_1.keys())

Index(['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'IAUID', 'type_x', 'Object', 'Confidence', 'p_SNII',
       'p_SNIa', 'p_SNIbc', 'type_y', 'Dec', 'RA', 'redshift'],
      dtype='object')


In [110]:
#type
merge_meta_1['type'] = merge_meta_1['type_x'].combine_first(merge_meta_1['type_y'])
merge_meta_1 = merge_meta_1.drop(columns=['type_x', 'type_y'])
print(merge_meta_1.keys())

Index(['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'IAUID', 'Object', 'Confidence', 'p_SNII', 'p_SNIa',
       'p_SNIbc', 'Dec', 'RA', 'redshift', 'type'],
      dtype='object')


let's keep only the relevant keys

In [111]:
rk=['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'IAUID', 'Confidence', 'p_SNII', 'p_SNIa',
       'p_SNIbc', 'Dec', 'RA', 'redshift', 'type']
merge_meta_1=merge_meta_1[rk]
merge_meta_1.shape

(9283, 27)

We still need to deal with issues 2 (RA/DEC units) and 3 (type inconsistencies):

In [121]:
#ra/dec to decimal:
def ra_to_decimal(ra_str):
    ra_parts = str(ra_str).split(':')
    if len(ra_parts)>1:
        ra_hours = float(ra_parts[0])
        ra_minutes = float(ra_parts[1])
        ra_seconds = float(ra_parts[2])
        ra_decimal = 15 * (ra_hours + ra_minutes / 60 + ra_seconds / 3600)
        return ra_decimal
    return float(ra_str)

In [124]:
def dec_to_decimal(dec_str):
    dec_parts = str(dec_str).split(':')
    if len(dec_parts)>1:
        dec_degrees = float(dec_parts[0])
        dec_minutes = float(dec_parts[1])
        dec_seconds = float(dec_parts[2])
        
        if dec_degrees < 0:
            dec_decimal = dec_degrees - dec_minutes / 60 - dec_seconds / 3600
        else:
            dec_decimal = dec_degrees + dec_minutes / 60 + dec_seconds / 3600
        return dec_decimal
    return float(dec_str)

In [122]:
merge_meta_1['RA_deg']=merge_meta_1['RA'].apply(lambda x: ra_to_decimal(x))

In [123]:
merge_meta_1.head()

,hostname,hostredshift,reporting_groups,data_sources,classifying_groups,associated_groups,ZTFID,peakt,peakfilt,peakmag,...,IAUID,Confidence,p_SNII,p_SNIa,p_SNIbc,Dec,RA,redshift,type,RA_deg
0,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaaoktp,NaN,NaN,NaN,...,SN2018mg,NaN,NaN,NaN,NaN,+79:16:42.50,10:56:53.580,0.110,SN Ia,164.223250
1,NaN,NaN,"ZTF, Pan-STARRS","ZTF, Pan-STARRS",ZTF,Pan-STARRS; ZTF,ZTF18aaaoaeq,NaN,NaN,NaN,...,SN2018mf,NaN,NaN,NaN,NaN,+10:26:24.09,10:22:09.778,0.107,SN Ia,155.540742
2,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaayemw,NaN,NaN,NaN,...,SN2018yt,NaN,NaN,NaN,NaN,+45:36:41.90,08:59:35.590,0.052,SN II,134.898292
3,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaewugy,NaN,NaN,NaN,...,SN2018bbk,NaN,NaN,NaN,NaN,+16:29:38.50,07:52:26.950,0.038,SN II,118.112292
4,NaN,0.0229,"ZTF, GaiaAlerts, ATLAS","ZTF, GaiaAlerts, ATLAS",ZTF,ATLAS; GaiaAlerts; ZTF,ZTF18aabssth,NaN,NaN,NaN,...,SN2018aex,NaN,NaN,NaN,NaN,+22:17:14.96,11:00:45.380,0.023,SN II,165.189083


In [125]:
merge_meta_1['Dec_deg']=merge_meta_1['Dec'].apply(lambda x: dec_to_decimal(x))

In [126]:
merge_meta_1.head()

,hostname,hostredshift,reporting_groups,data_sources,classifying_groups,associated_groups,ZTFID,peakt,peakfilt,peakmag,...,Confidence,p_SNII,p_SNIa,p_SNIbc,Dec,RA,redshift,type,RA_deg,Dec_deg
0,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaaoktp,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,+79:16:42.50,10:56:53.580,0.110,SN Ia,164.223250,79.278472
1,NaN,NaN,"ZTF, Pan-STARRS","ZTF, Pan-STARRS",ZTF,Pan-STARRS; ZTF,ZTF18aaaoaeq,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,+10:26:24.09,10:22:09.778,0.107,SN Ia,155.540742,10.440025
2,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaayemw,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,+45:36:41.90,08:59:35.590,0.052,SN II,134.898292,45.611639
3,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaewugy,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,+16:29:38.50,07:52:26.950,0.038,SN II,118.112292,16.494028
4,NaN,0.0229,"ZTF, GaiaAlerts, ATLAS","ZTF, GaiaAlerts, ATLAS",ZTF,ATLAS; GaiaAlerts; ZTF,ZTF18aabssth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,+22:17:14.96,11:00:45.380,0.023,SN II,165.189083,22.287489


In [128]:
merge_meta_1 = merge_meta_1.drop(columns=['Dec', 'RA'])

In [129]:
merge_meta_1.shape

(9283, 27)

Now let's see what's up with the types

In [131]:
merge_meta_1.type.unique()

array(['SN Ia', 'SN II', 'SN IIn', 'SN IIP', 'SN Ic-BL', 'SLSN-II',
       'SN Ib', 'SLSN-I', 'SN Ib/c', 'SN IIb', 'SN Ib-Ca-rich', 'SN Ibn',
       'SN IIL', 'SN II-pec', 'SN Icn', 'SN Ib-pec', 'SN Ic',
       'SN IIn-pec', 'SN Ibn/Icn', 'SN Ia-pec', 'SN Ia-91T', 'SN Ia-91bg',
       'SN Iax', 'SN Ia-CSM', 'SN Ia-SC', 'SN Ic-pec', 'SNIa-norm',
       'SNIbc', 'SNII', 'SNIa', 'SNIc', 'SNIb', 'SNIbn', 'SNIa-91T-like',
       'SNIa-CSM', 'SN', 'SNIa-SC', 'SNIc-BL', 'SNIIb', 'SNIax', 'TDE',
       'SNIIn', 'LBV', 'LRN', 'SNIa-91bg-like'], dtype=object)

Ok, so we need to do the following:

1.-drop everything that isn't a SN

2.-get rid of spaces ' '

3.-rename the column to subtype

4.-add broader types: Ia, Ib/c, II and SLSN

5.-add numeric code to types

In [133]:
#drop non SN objects
merge_meta_sn = merge_meta_1[merge_meta_1.type.str.contains('SN')]

In [134]:
merge_meta_sn.shape

(9278, 27)

In [135]:
merge_meta_sn.type.unique()

array(['SN Ia', 'SN II', 'SN IIn', 'SN IIP', 'SN Ic-BL', 'SLSN-II',
       'SN Ib', 'SLSN-I', 'SN Ib/c', 'SN IIb', 'SN Ib-Ca-rich', 'SN Ibn',
       'SN IIL', 'SN II-pec', 'SN Icn', 'SN Ib-pec', 'SN Ic',
       'SN IIn-pec', 'SN Ibn/Icn', 'SN Ia-pec', 'SN Ia-91T', 'SN Ia-91bg',
       'SN Iax', 'SN Ia-CSM', 'SN Ia-SC', 'SN Ic-pec', 'SNIa-norm',
       'SNIbc', 'SNII', 'SNIa', 'SNIc', 'SNIb', 'SNIbn', 'SNIa-91T-like',
       'SNIa-CSM', 'SN', 'SNIa-SC', 'SNIc-BL', 'SNIIb', 'SNIax', 'SNIIn',
       'SNIa-91bg-like'], dtype=object)

In [136]:
#2 get rid of spa
merge_meta_sn['subtype']= merge_meta_sn['type'].apply(lambda x: x.replace(' ',''))

/tmp/ipykernel_12943/155868285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_meta_sn['subtype']= merge_meta_sn['type'].apply(lambda x: x.replace(' ',''))


In [137]:
merge_meta_sn.head()

,hostname,hostredshift,reporting_groups,data_sources,classifying_groups,associated_groups,ZTFID,peakt,peakfilt,peakmag,...,IAUID,Confidence,p_SNII,p_SNIa,p_SNIbc,redshift,type,RA_deg,Dec_deg,subtype
0,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaaoktp,NaN,NaN,NaN,...,SN2018mg,NaN,NaN,NaN,NaN,0.110,SN Ia,164.223250,79.278472,SNIa
1,NaN,NaN,"ZTF, Pan-STARRS","ZTF, Pan-STARRS",ZTF,Pan-STARRS; ZTF,ZTF18aaaoaeq,NaN,NaN,NaN,...,SN2018mf,NaN,NaN,NaN,NaN,0.107,SN Ia,155.540742,10.440025,SNIa
2,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaayemw,NaN,NaN,NaN,...,SN2018yt,NaN,NaN,NaN,NaN,0.052,SN II,134.898292,45.611639,SNII
3,NaN,NaN,ZTF,ZTF,ZTF,ZTF,ZTF18aaewugy,NaN,NaN,NaN,...,SN2018bbk,NaN,NaN,NaN,NaN,0.038,SN II,118.112292,16.494028,SNII
4,NaN,0.0229,"ZTF, GaiaAlerts, ATLAS","ZTF, GaiaAlerts, ATLAS",ZTF,ATLAS; GaiaAlerts; ZTF,ZTF18aabssth,NaN,NaN,NaN,...,SN2018aex,NaN,NaN,NaN,NaN,0.023,SN II,165.189083,22.287489,SNII


In [138]:
merge_meta_sn.subtype.unique()

array(['SNIa', 'SNII', 'SNIIn', 'SNIIP', 'SNIc-BL', 'SLSN-II', 'SNIb',
       'SLSN-I', 'SNIb/c', 'SNIIb', 'SNIb-Ca-rich', 'SNIbn', 'SNIIL',
       'SNII-pec', 'SNIcn', 'SNIb-pec', 'SNIc', 'SNIIn-pec', 'SNIbn/Icn',
       'SNIa-pec', 'SNIa-91T', 'SNIa-91bg', 'SNIax', 'SNIa-CSM',
       'SNIa-SC', 'SNIc-pec', 'SNIa-norm', 'SNIbc', 'SNIa-91T-like', 'SN',
       'SNIa-91bg-like'], dtype=object)

In [139]:
merge_meta_sn[merge_meta_sn.type == 'SN']

,hostname,hostredshift,reporting_groups,data_sources,classifying_groups,associated_groups,ZTFID,peakt,peakfilt,peakmag,...,IAUID,Confidence,p_SNII,p_SNIa,p_SNIbc,redshift,type,RA_deg,Dec_deg,subtype
7849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SN2020apw,0.998,0.002,0.998,0.000,0.224,SN,141.012066,28.814982,SN
7853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SN2020awu,0.975,0.975,0.018,0.007,0.087,SN,171.348033,9.983899,SN
8908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SN2021kqp,0.981,0.000,0.981,0.019,0.095,SN,223.794754,-6.985927,SN
8914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SN2021kyv,0.997,0.997,0.002,0.001,0.105,SN,216.980358,33.002080,SN


we'll need to fix that. didn't see coming that some spec types would be plain SN

In [144]:
merge_meta_sn.loc[merge_meta_sn['IAUID']=='SN2020apw','subtype']='SNIa'
merge_meta_sn.loc[merge_meta_sn['IAUID']=='SN2020awu','subtype']='SNII'
merge_meta_sn.loc[merge_meta_sn['IAUID']=='SN2021kqp','subtype']='SNIa'
merge_meta_sn.loc[merge_meta_sn['IAUID']=='SN2021kyv','subtype']='SNII'

In [146]:
merge_meta_sn.subtype.unique()

array(['SNIa', 'SNII', 'SNIIn', 'SNIIP', 'SNIc-BL', 'SLSN-II', 'SNIb',
       'SLSN-I', 'SNIb/c', 'SNIIb', 'SNIb-Ca-rich', 'SNIbn', 'SNIIL',
       'SNII-pec', 'SNIcn', 'SNIb-pec', 'SNIc', 'SNIIn-pec', 'SNIbn/Icn',
       'SNIa-pec', 'SNIa-91T', 'SNIa-91bg', 'SNIax', 'SNIa-CSM',
       'SNIa-SC', 'SNIc-pec', 'SNIa-norm', 'SNIbc', 'SNIa-91T-like',
       'SNIa-91bg-like'], dtype=object)

Now let's replace 'type'column so it contains broad types

In [153]:
def broad_type(t):
    t=str(t)
    if 'Ia' in t:
        t='Ia'
    elif 'Ib' in t or 'Ic' in t:
        t='Ib/c'
    elif 'II' in t:
        t='II'
    elif 'SL' in t:
        t='SLSN'
    else:
        print(t)
        return t
    return t

In [154]:
merge_meta_sn['type_broad']=merge_meta_sn['type'].apply(lambda x: broad_type(x))

/tmp/ipykernel_12943/3401721392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_meta_sn['type_broad']=merge_meta_sn['type'].apply(lambda x: broad_type(x))


In [156]:
merge_meta_sn['type_broad'].unique()

array(['Ia', 'II', 'Ib/c', 'SLSN'], dtype=object)

In [157]:
merge_meta_sn = merge_meta_sn.drop(columns=['type'])
merge_meta_sn.rename(columns={'type_broad':'type'},inplace=True)

In [158]:
merge_meta_sn.keys()

Index(['hostname', 'hostredshift', 'reporting_groups', 'data_sources',
       'classifying_groups', 'associated_groups', 'ZTFID', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'hostabs', 'hostcol',
       'b', 'A_V', 'IAUID', 'Confidence', 'p_SNII', 'p_SNIa', 'p_SNIbc',
       'redshift', 'RA_deg', 'Dec_deg', 'subtype', 'type'],
      dtype='object')

In [159]:
merge_meta_sn.shape

(9278, 28)

In [160]:
merge_meta_sn.IAUID.unique().shape

(9278,)

In [161]:
def add_numeric_codes(t):
    if t=='Ia':
        return 0
    elif t=='Ib/c':
        return 1
    elif t=='II':
        return 2
    elif t=='SLSN':
        return 3

In [162]:
merge_meta_sn['type_code']=merge_meta_sn['type'].apply(lambda x : add_numeric_codes(x))

In [163]:
merge_meta_sn.shape

(9278, 29)

In [164]:
merge_meta_sn.type_code.unique()

array([0, 2, 1, 3])

now let's save the metadata file

In [165]:
merge_meta_sn.to_csv('../data/real/real_metadata.csv',index=False)